# Part X — Advanced Topics  
## 46. Advanced Testing and Quality Engineering (Property‑Based, Contract, Load, Quality Gates)

This chapter upgrades your testing from:
- “we have tests”
to
- “we have **quality engineering**.”

That means:
- preventing whole classes of bugs (not just known ones)
- enforcing contracts (API schemas, invariants, permissions)
- catching performance regressions early
- validating real-world behavior under load
- building CI quality gates that match industry expectations

We’ll keep everything tied to what you already built:
- Articles + comments + moderation
- Orgs + memberships + tasks + exports
- DRF APIs + webhooks + background jobs
- Channels/WebSockets (optional)

---

## 46.0 Learning Outcomes

By the end of this chapter, you should be able to:

1. Use **property-based testing** to find edge cases automatically.
2. Write **contract tests** for your API using OpenAPI (schema-driven tests).
3. Add **load tests** (not unit tests) to verify performance and capacity.
4. Define and enforce **quality gates**:
   - coverage minimums
   - query-count budgets
   - performance budgets (p95 latency targets)
   - “no missing migrations”
   - dependency audit and basic security checks
5. Reduce test flakiness by controlling time, randomness, concurrency, and external calls.
6. Create a “quality plan” document that says what you test, how, and why.

---

# 46.1 Quality Engineering Mindset (What Changes at This Level)

### 46.1.1 Unit vs integration vs “quality engineering”
You already learned unit/integration/E2E. Quality engineering adds:
- invariant testing (property-based)
- schema contract testing (API)
- non-functional testing (load, performance, reliability)
- gates that prevent regressions before deployment

### 46.1.2 What you’re trying to prevent
- silent authorization drift (a refactor changes permissions)
- subtle validation bugs (empty strings, unicode, extreme lengths)
- serialization errors (missing keys, wrong types)
- performance regressions (N+1, slow query)
- infrastructure regressions (migrations missing, settings broken)

---

# 46.2 Property‑Based Testing (PBT) with Hypothesis

Property-based testing means:
- you define **properties** that must always hold
- the tool generates many randomized inputs to try to break your code
- it shrinks failures to a minimal counterexample

This is extremely good for:
- parsers (query params, slugs)
- validators (forms/serializers)
- policy logic (authorization invariants)
- webhook signature logic
- caching key builders (must not collide across tenants)

## 46.2.1 Install Hypothesis (dev dependency)
Add to `requirements-dev.txt`:

```text
hypothesis
```

Install:

```bash
python -m pip install -r requirements-dev.txt
```

---

## 46.2.2 Example 1: Property test for your tag slug normalization

Assume you have logic like:
- `ArticleWriteSerializer.validate_tags` slugifies tags
- dedupes them
- enforces max count

We’ll test these properties:

1. Output contains only slug-safe patterns (letters, digits, hyphens).
2. Output has no duplicates.
3. Output length <= 20.
4. Empty / whitespace-only tags are rejected.

Create `tests/test_article_tag_properties.py`:

```python
import re

import pytest
from hypothesis import given, strategies as st
from rest_framework.test import APIRequestFactory

from articles.api.serializers_write import ArticleWriteSerializer


_slug_re = re.compile(r"^[a-z0-9]+(?:-[a-z0-9]+)*$")


def make_serializer():
    rf = APIRequestFactory()
    request = rf.post("/api/v1/articles/", data={})
    request.user = type(
        "U",
        (),
        {"is_authenticated": True, "is_staff": True, "id": 1, "email": "x@y.com"},
    )()
    return ArticleWriteSerializer(context={"request": request})


@given(st.lists(st.text(min_size=0, max_size=30), min_size=0, max_size=60))
def test_validate_tags_dedupes_and_limits(tags):
    s = make_serializer()

    try:
        cleaned = s.validate_tags(tags)
    except Exception:
        # If it raises, that's acceptable for some inputs (e.g., empty tags).
        # But we still want to ensure it never returns invalid output.
        return

    assert len(cleaned) <= 20
    assert len(cleaned) == len(set(cleaned))

    for slug in cleaned:
        assert _slug_re.match(slug), f"Not a slug: {slug!r}"


@given(st.lists(st.just("   "), min_size=1, max_size=5))
def test_validate_tags_rejects_whitespace_only(tags):
    s = make_serializer()
    with pytest.raises(Exception):
        s.validate_tags(tags)
```

### Why this is valuable
You’ll catch weird corner cases like:
- unicode spaces
- empty strings
- repeated values in different cases (“Django”, “django”, “ DJANGO ”)
- long tag lists that could be abused

And Hypothesis will shrink failures to minimal inputs.

---

## 46.2.3 Example 2: Property test for policy invariants (authorization)

We want to ensure:
- Org ADMIN can always edit tasks in their org.
- MEMBER can edit iff they are creator or assignee (per your policy).
- Nobody can edit tasks outside their org (tenant scoping must stop that at query level; but the policy should still behave sensibly).

Create `tests/test_task_policy_properties.py`:

```python
from hypothesis import given, strategies as st

from core.policy import Decision
from orgs.models import Membership
from tasks.policies import can_edit_task


class Dummy:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)


@given(
    actor_id=st.integers(min_value=1, max_value=10_000),
    creator_id=st.integers(min_value=1, max_value=10_000),
    assignee_id=st.one_of(st.none(), st.integers(min_value=1, max_value=10_000)),
)
def test_member_edit_rule_equivalence(actor_id, creator_id, assignee_id):
    membership = Dummy(role=Membership.Role.MEMBER)
    actor = Dummy(id=actor_id, is_authenticated=True)
    task = Dummy(created_by_id=creator_id, assigned_to_id=assignee_id)

    decision = can_edit_task(membership=membership, actor=actor, task=task)

    expected = actor_id == creator_id or actor_id == assignee_id
    assert decision.allowed == expected


@given(
    actor_id=st.integers(min_value=1, max_value=10_000),
    creator_id=st.integers(min_value=1, max_value=10_000),
    assignee_id=st.one_of(st.none(), st.integers(min_value=1, max_value=10_000)),
)
def test_admin_always_allows_edit(actor_id, creator_id, assignee_id):
    membership = Dummy(role=Membership.Role.ADMIN)
    actor = Dummy(id=actor_id, is_authenticated=True)
    task = Dummy(created_by_id=creator_id, assigned_to_id=assignee_id)

    decision = can_edit_task(membership=membership, actor=actor, task=task)
    assert decision.decision == Decision.ALLOW
```

### Why this is valuable
Policy code tends to rot as rules grow. Property tests enforce the *shape of the
rule*, catching subtle regressions.

---

## 46.2.4 Example 3: Property test for webhook signature verification

You already built:
- `compute_signature(secret, timestamp, body)`
- `verify_signature(...)`

Key property:
- verify(compute(...)) should always pass.

Create `tests/test_webhook_signature_properties.py`:

```python
import time

from hypothesis import given, strategies as st

from integrations.webhooks.security import compute_signature, verify_signature


@given(
    secret=st.text(min_size=1, max_size=50),
    body=st.binary(min_size=0, max_size=500),
)
def test_webhook_signature_round_trip(secret, body):
    timestamp = str(int(time.time()))
    sig = compute_signature(secret=secret, timestamp=timestamp, body=body)

    # If this raises, the property is broken.
    verify_signature(
        secret=secret,
        timestamp=timestamp,
        signature=sig,
        body=body,
        tolerance_seconds=10**9,
    )
```

### Why this is valuable
It catches encoding mistakes and signature formatting drift early.

---

# 46.3 Contract Testing for APIs (OpenAPI‑Driven)

Contract testing means:
- your API publishes a schema (“this is what endpoints accept/return”)
- tests automatically verify endpoints match that schema

This catches:
- accidental field removals
- wrong status codes
- wrong response shapes
- missing required fields
- inconsistent error envelopes

There are two common approaches:

1. **Schema validation tests** (your responses validated against OpenAPI).
2. **Schema-driven fuzzing** (tool generates requests to try edge cases).

If you already added OpenAPI docs using drf-spectacular (Chapter 26), you’re ready.

---

## 46.3.1 Ensure you can generate OpenAPI schema in CI
You should have:
- `/api/schema/` endpoint (OpenAPI JSON)

Test that it returns 200 in CI (smoke):

```python
from django.test import TestCase
from django.urls import reverse


class OpenApiSchemaTests(TestCase):
    def test_schema_endpoint(self):
        # Adjust if your schema route name differs
        response = self.client.get("/api/schema/")
        self.assertEqual(response.status_code, 200)
        self.assertIn("openapi", response.json())
```

---

## 46.3.2 Schema-driven tests with Schemathesis (high-value)
Schemathesis (commonly used) can:
- read OpenAPI schema
- generate requests (valid and invalid)
- check responses against schema

Add to `requirements-dev.txt`:

```text
schemathesis
```

Install:

```bash
python -m pip install -r requirements-dev.txt
```

### 46.3.2.1 Run against your running server (staging or local)
Start server (or use a CI job that runs it), then:

```bash
schemathesis run http://127.0.0.1:8000/api/schema/
```

### What to expect
- It will try many combinations of parameters and payloads.
- It will likely find issues the first time:
  - missing schema definitions for some endpoints
  - inconsistent error shapes
  - endpoints returning HTML on error
These are precisely the issues contract testing is designed to catch.

**Industry practice:** run schema fuzzing:
- nightly, or
- on main branch, or
- on staging pipeline  
because it can be heavier than normal unit tests.

---

## 46.3.3 Contract tests for error envelopes (if you customized DRF exceptions)
If you implemented a custom exception handler that returns:

```json
{"error": {"code": "...", "message": "...", "details": ...}}
```

Write a contract test that enforces it:

```python
from rest_framework.test import APITestCase


class ApiErrorEnvelopeTests(APITestCase):
    def test_unauthenticated_write_has_error_envelope(self):
        r = self.client.post("/api/v1/orgs/acme/tasks/", data={"title": "x"})
        self.assertIn(r.status_code, {401, 403})
        payload = r.json()
        self.assertIn("error", payload)
        self.assertIn("code", payload["error"])
        self.assertIn("message", payload["error"])
        self.assertIn("details", payload["error"])
```

This prevents “someone removed exception handler and broke frontend.”

---

# 46.4 Load Testing (Non‑Functional Tests)

Load testing is not unit testing. It answers:
- how many concurrent users you can handle
- what your p95/p99 latency is
- where bottlenecks appear (DB, cache, CPU)
- whether your worker pool saturates
- whether Nginx / proxy settings are correct
- whether timeouts are correct

**Never run load tests on production without planning.**  
Use staging or a dedicated perf environment.

---

## 46.4.1 Pick a load testing tool (industry common)
- **Locust** (Python-based, very popular with Django teams)
- **k6** (JS-based, high-performance)
- **wrk** (simple HTTP benchmarking)
- **ab** (ApacheBench; older)

We’ll implement Locust because it integrates nicely with Django devs.

---

## 46.4.2 Install Locust
Add to `requirements-dev.txt`:

```text
locust
```

Install:

```bash
python -m pip install -r requirements-dev.txt
```

---

## 46.4.3 Create a Locust file that hits your real endpoints
Create `locustfile.py`:

```python
from locust import HttpUser, between, task


class PublicUser(HttpUser):
    wait_time = between(1, 3)

    @task(5)
    def articles_list(self):
        self.client.get("/articles/")

    @task(3)
    def articles_search(self):
        self.client.get("/articles/?q=django")

    @task(2)
    def api_articles(self):
        self.client.get("/api/v1/articles/")


class OrgUser(HttpUser):
    wait_time = between(1, 2)

    def on_start(self):
        # If you have a token auth endpoint, obtain a token here and store it.
        # Otherwise, this user will behave as anonymous.
        pass

    @task(5)
    def tasks_list(self):
        self.client.get("/orgs/acme/tasks/")

    @task(1)
    def readyz(self):
        self.client.get("/readyz/")
```

### How to run Locust
Start your app (staging/local with realistic DB size), then:

```bash
locust -H http://127.0.0.1:8000
```

Open the Locust UI in browser, set:
- number of users
- spawn rate
- run time

### What to look at
- p95 latency for each endpoint
- failure rate (non-2xx)
- throughput (requests/sec)
- whether DB CPU spikes
- whether `/readyz` stays stable

---

## 46.4.4 Load testing authenticated APIs (token auth recommended)
If you implemented TokenAuthentication:

Update `OrgUser.on_start()` to obtain token:

```python
class OrgUser(HttpUser):
    wait_time = between(1, 2)

    def on_start(self):
        r = self.client.post(
            "/api/v1/auth/token/",
            json={"username": "loaduser", "password": "pass12345"},
        )
        token = r.json()["token"]
        self.client.headers.update({"Authorization": f"Token {token}"})
```

Then test API endpoints realistically:

```python
    @task(3)
    def api_tasks_list(self):
        self.client.get("/api/v1/orgs/acme/tasks/")
```

**Industry warning:** creating tokens/users dynamically in load tests can distort results; pre-provision a load test user and data.

---

## 46.4.5 Performance budgets (turn load testing into gates)
Define budgets like:
- `/articles/` p95 < 300ms at 50 concurrent users
- `/api/v1/articles/` p95 < 200ms
- `/orgs/<slug>/tasks/` p95 < 400ms

You don’t need perfect numbers; you need:
- a baseline
- alerts when regression is significant

---

# 46.5 Quality Gates (CI Rules That Prevent Regressions)

Quality gates are checks that must pass to merge/deploy.

A mature Django project typically has gates for:

## 46.5.1 Code quality gates
- Ruff: `python -m ruff check .`
- Black check: `python -m black . --check`
- No missing migrations: `python manage.py makemigrations --check --dry-run`

## 46.5.2 Test gates
- `pytest` (or manage.py test)
- coverage minimum (e.g., 80% for core apps)

Example pytest-cov gate:

```bash
python -m pytest --cov=. --cov-fail-under=80
```

## 46.5.3 Performance regression gates
Two high-value ones:
1. Query count budgets on hot endpoints (`assertNumQueries`)
2. “No N+1” tests for list pages and serializers

Example (pytest + django_assert_num_queries) you already used.

## 46.5.4 Contract gates
- schema endpoint must generate
- basic schema validation tests pass
- optionally schemathesis run on staging/nightly

## 46.5.5 Security gates (minimal baseline)
- dependency audit (`pip-audit`)
- ensure DEBUG isn’t on in prod settings
- `python manage.py check --deploy` must pass in CI (with CI settings)

---

# 46.6 Reducing Flaky Tests (Production Teams Care About This a Lot)

Flaky tests waste time and reduce trust. Common causes:

### 46.6.1 Time and time zones
Fix:
- freeze time (`freezegun`)
- use `timezone.now()` consistently
- avoid “assert exact timestamp string” unless frozen

### 46.6.2 Randomness
Fix:
- seed randomness where needed
- Hypothesis already shrinks cases; keep deterministic DB state

### 46.6.3 External network calls
Fix:
- block network in tests
- use `responses` or mocks
- ensure Celery tasks eager mode or patched

### 46.6.4 Concurrency
Fix:
- avoid real background threads in tests
- test concurrency logic via smaller pure functions where possible

---

# 46.7 API Contract + Tenant Isolation = High‑Value Combined Tests

A professional regression suite for your multi-tenant API includes:

1. Schema tests (docs generated)
2. Auth tests (401/403)
3. Tenant isolation tests (outsider cannot access)
4. Contract shape tests (consistent error envelope)
5. Query count tests (list endpoints stable under many objects)

This combination catches the highest-impact bugs before production.

---

# 46.8 Chapter Capstone Lab (Do This to Lock It In)

## Lab A — Add Hypothesis tests
1. Add 2 property-based tests:
   - tag slug normalization properties
   - task policy equivalence properties

## Lab B — Add contract smoke tests
1. Add a test that `/api/schema/` returns OpenAPI JSON.
2. Add a test that errors are wrapped in your error envelope.

## Lab C — Add a performance budget test
1. Add a query-count test for:
   - article API list endpoint
   - tasks list page (org-scoped)

## Lab D — Create a load testing plan (document)
1. Write `docs/performance.md`:
   - target endpoints
   - expected p95 budgets
   - dataset size assumptions
   - how to run Locust
2. Run a small load test on staging/local with seeded data.

---

## 46.9 Exercises (Do These Before Proceeding)

1. Add property-based tests for your webhook dedupe rule:
   - event id uniqueness constraint must prevent duplicates (in DB test).
2. Add a schemathesis run in CI as a nightly job (not on every PR).
3. Add a “no network” rule in tests:
   - ensure any accidental `requests.get` fails the test (mock or policy).
4. Add a load test that simulates:
   - 80% reads (list/detail)
   - 20% writes (status updates)
   - measure error rate and p95

---

## 46.10 Chapter Summary

- Property-based tests find edge cases you didn’t think to write.
- Contract testing prevents silent API drift and keeps frontends stable.
- Load testing validates non-functional behavior (latency, throughput, failure modes).
- Quality gates in CI prevent regressions in:
  - correctness
  - migrations
  - performance
  - API contracts
  - dependencies/security baseline

---

Next chapter: **47. Managing Large Codebases**  
We’ll cover modularization, app boundaries, internal packages, architectural rules,
feature flags, dependency management at scale, and how to enforce structure in a
multi-developer Django monolith.